In [ ]:
import findspark
import pickle
findspark.init()

In [ ]:
#!sudo netstat -tlnp|grep 5050

In [ ]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys 
import requests
import re
import socket
import time as time1

In [ ]:
sc = SparkContext("local[2]", "NetworkWordCount")
ssc = StreamingContext(sc, 1)
dataStream=ssc.socketTextStream("localhost",9009)

In [ ]:
tags = ['#louwilliams', '#stephencurry', '#kevindurant', '#cousins', 'williams', 'curry', 'durant', 'demarcus']

In [ ]:
TCP_IP = "localhost"
TCP_PORT = 5050
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()

In [ ]:
def gethash(tweet_text):
    words = tweet_text.split()
    hashtags = [w.lower() for w in words if w.lower() in tags]
    hashtags = [[w,tags[tags.index(w)-4]][tags.index(w)>3] for w in hashtags]
    return True if hashtags else False

In [ ]:
def preprocessing(original_tweet_text):
    words = original_tweet_text.split()
    hashtags = [w.lower() for w in words if w.lower() in tags]
    hashtags = [[w,tags[tags.index(w)-4]][tags.index(w)>3] for w in hashtags]
    hashtags = ' '.join(hashtags)
    tweet_text = ' '.join([words, words[1:]][words[0]=='RT'])
    tweet_text=re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', tweet_text, flags=re.MULTILINE)
    tweet_text=re.sub('@[\w]*', ' ', tweet_text)
    tweet_text=re.sub('#[\w]*', ' ', tweet_text)
    tweet_text=re.sub('[^A-Za-z0-9]+', ' ', tweet_text)
    return (hashtags,original_tweet_text,tweet_text)

In [ ]:
def get_sql_context_instance(spark_context):
    if('sqlContextSingletonInstance'not in globals()):
        globals()['sqlContextSingletonInstance']=SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']

def process_rdd(time,rdd):
    print("-----------%s-----------"%str(time))
    try:
        sql_context=get_sql_context_instance(rdd.context)
        row_rdd = rdd.map(lambda w: Row(hashtag=w[0],original=w[1],text=w[2]))
        df = sql_context.createDataFrame(row_rdd)
    except:
        e=sys.exc_info()[0]
        print(e)
    finally:
        try:
            if row_rdd.take(1):
                df1 = df.toPandas()
                #msg = (df1['hashtag'].values,df1['text'].values)
                #msg = (df['hashtag'].values,df['text'].values)
                #print(msg)
                start = time1.time()
                for name, original, text in df1.values:
                    msg = {'name':name,'original':original,'text':text,'time':start}
                    #print(msg)
                    print('conn ready')
                    conn.send(pickle.dumps(msg))
                    print('conn successful')


        except Exception as e:
            print(e)

In [ ]:
#split each tweet into words
#words=dataStream.flatMap(lambda line:line.split(" "))
words = dataStream.filter(gethash).map(preprocessing)

In [ ]:
#words.pprint()

In [ ]:
words.foreachRDD(process_rdd)

In [ ]:
#start the streaming computations
ssc.start()
#wait for the streaming to finish
ssc.awaitTermination()